# All Weather Strategy: v1.0 vs v1.1 vs v1.2 Comparison

Comprehensive comparison of three versions of the All Weather Strategy:

- **v1.0**: Pure risk parity (always rebalance, sample covariance)
- **v1.1**: + Adaptive rebalancing (5% drift threshold)
- **v1.2**: + Ledoit-Wolf covariance shrinkage

**Period**: 2018-01-01 to 2026-01-28 (8 years)

**Goal**: Measure the incremental impact of each improvement on performance, stability, and costs.

---

## What is Risk Parity?

Risk parity is a portfolio allocation strategy where **each asset contributes equally to total portfolio risk**.

**Mathematical Definition:**

For asset $i$, its risk contribution is:

$$RC_i = w_i \cdot \frac{(\Sigma w)_i}{\sigma_p}$$

Where:
- $w_i$ = weight of asset $i$
- $\Sigma$ = covariance matrix
- $\sigma_p = \sqrt{w^T \Sigma w}$ = portfolio volatility

**Goal**: $RC_1 = RC_2 = ... = RC_n$ (equal risk contribution)

This is achieved by minimizing: $\text{std}(RC_1, RC_2, ..., RC_n)$

**Implementation**: See `src/optimizer.py:14-78` (function `optimize_weights`)

## 1. Setup & Imports

### What We're Doing:
Loading necessary Python libraries and our custom modules for strategy comparison.

### Key Imports:
- **`VersionComparison`** (`src/comparison.py:35-290`): Main class that orchestrates running backtests for multiple strategy versions and comparing results
- **`VERSION_CONFIGS`** (`src/comparison.py:23-32`): Predefined configurations for v1.0, v1.1, v1.2
- **`estimate_covariance_shrinkage`** (`src/optimizer.py:15-30`): Ledoit-Wolf shrinkage implementation
- **Reporting utilities** (`src/utils/reporting.py`): Functions to format and display results consistently

### Before Moving On:
1. Review `src/comparison.py:23-32` to see how versions are defined
2. Glance at `src/comparison.py:35-60` to understand the `VersionComparison` class constructor

In [ ]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Import modular components
from src.data_loader import load_prices
from src.comparison import VersionComparison, VERSION_CONFIGS
from src.optimizer import estimate_covariance_shrinkage, risk_contribution
from src.utils.reporting import print_section, format_currency, format_percentage

# Plotting style
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14, 7)
plt.rcParams['font.size'] = 10

# Display options
pd.options.display.float_format = '{:.4f}'.format

print("✓ Imports successful")

## 2. Load Data

### What We're Doing:
Loading historical price data for 7 Chinese ETFs covering stocks, bonds, and commodities.

### ETF Universe (from `data/etf_prices_7etf.csv`):
1. **510300.SH**: CSI 300 Index (Large-cap stocks)
2. **510500.SH**: CSI 500 Index (Mid/small-cap stocks)
3. **513500.SH**: S&P 500 (US stocks)
4. **511260.SH**: 10-Year Treasury Bonds
5. **518880.SH**: Gold
6. **000066.SH**: China Index
7. **513100.SH**: Nasdaq-100 (US tech stocks)

### Data Quality:
This is the **7-ETF aligned dataset** (highest quality):
- All ETFs have <5% zero returns (excellent liquidity)
- No frozen price issues
- Complete data from 2015-01-05 to 2026-01-28

**Implementation**: See `src/data_loader.py:18-34` (function `load_prices`)

### Before Moving On:
Review `data/DATA_QUALITY_REPORT.md` to understand data quality metrics

In [ ]:
# Load 7-ETF aligned dataset (highest quality)
prices = load_prices('../data/etf_prices_7etf.csv')

print(f"Data shape: {prices.shape}")
print(f"Date range: {prices.index[0]} to {prices.index[-1]}")
print(f"ETFs: {list(prices.columns)}")
print(f"\nMissing values: {prices.isnull().sum().sum()}")

prices.tail()

## 3. Version Overview

### What We're Doing:
Displaying the configuration parameters that differentiate each version.

### Version Differences:

| Version | Rebalance Threshold | Use Shrinkage | Key Innovation |
|---------|-------------------|---------------|----------------|
| v1.0 | 0% (always) | No | Pure risk parity baseline |
| v1.1 | 5% (adaptive) | No | Reduce transaction costs |
| v1.2 | 5% (adaptive) | Yes | Robust covariance estimation |

### Adaptive Rebalancing (v1.1):

Only rebalance when weight drift exceeds threshold:

$$\text{drift} = \max_i |w_i^{\text{current}} - w_i^{\text{target}}|$$

If drift > 5%, rebalance. Otherwise, skip.

**Why?** Reduces unnecessary trading when weights are stable.

**Implementation**: See `src/strategy.py:61-90` (method `should_rebalance`)

### Covariance Shrinkage (v1.2):

Ledoit-Wolf shrinkage combines sample covariance with structured estimator:

$$\Sigma_{\text{shrunk}} = (1-\lambda) \Sigma_{\text{sample}} + \lambda \Sigma_{\text{target}}$$

Where $\lambda \in [0, 1]$ is automatically chosen to minimize expected estimation error.

**Why?** Sample covariance is noisy with limited data. Shrinkage reduces noise.

**Implementation**: See `src/optimizer.py:15-30` (function `estimate_covariance_shrinkage`)

### Before Moving On:
1. Read `src/comparison.py:23-32` to see the exact parameters
2. Review `src/strategy.py:15-50` to understand the strategy class initialization

In [ ]:
# Display version configurations
print_section("Version Configurations")

for version_name, config in VERSION_CONFIGS.items():
    print(f"\n{config.name}: {config.description}")
    print(f"  Rebalance threshold: {config.rebalance_threshold:.0%}")
    print(f"  Use shrinkage: {config.use_shrinkage}")

## 4. Run Backtests

### What We're Doing:
Running historical simulations for all three versions using the same data and parameters.

### Backtest Process (for each version):

1. **Initialize Portfolio**: Start with ¥1,000,000 cash
2. **For each trading day**:
   - Calculate current portfolio value
   - If it's a rebalance date (Monday):
     a. Calculate returns from last 252 days (lookback window)
     b. Estimate covariance matrix (with/without shrinkage)
     c. Optimize weights for equal risk contribution
     d. Check if drift > threshold (v1.1, v1.2 only)
     e. Execute trades if needed (with 0.03% commission)
3. **Track**: Equity curve, returns, weights, commissions

### Mathematical Flow:

$$\text{Returns}_{t-252:t} \rightarrow \Sigma \rightarrow \min \text{std}(RC_1, ..., RC_n) \rightarrow w^*$$

### Implementation Details:
- **Main backtest loop**: `src/comparison.py:62-95` (method `run_version`)
- **Strategy backtest**: `src/strategy.py:93-160` (method `run_backtest`)
- **Portfolio rebalancing**: `src/portfolio.py:94-217` (method `rebalance`)
- **Weight optimization**: `src/optimizer.py:33-78` (function `optimize_weights`)

### Parameters:
- **Initial capital**: ¥1,000,000
- **Start date**: 2018-01-01 (needs 252 days lookback before this)
- **Lookback**: 252 days (≈1 year of trading days)
- **Commission**: 0.03% per trade
- **Rebalance frequency**: Weekly (every Monday)

### Before Moving On:
1. Read `src/comparison.py:62-95` to understand the backtest orchestration
2. Review `src/strategy.py:93-160` to see the detailed backtest logic
3. Check `src/optimizer.py:33-78` to understand weight optimization

In [ ]:
# Initialize comparison
comparison = VersionComparison(
    prices=prices,
    initial_capital=1_000_000,
    start_date='2018-01-01',
    lookback=252,
    commission_rate=0.0003
)

# Run all versions
comparison.run_all_versions(verbose=True)

## 5. Performance Comparison

### What We're Doing:
Displaying a comprehensive table comparing all versions across key performance metrics.

### Metrics Explained:

**Return Metrics:**
- **Total Return**: $(\text{Final Value} - \text{Initial}) / \text{Initial}$
- **Annual Return**: Geometric mean annualized: $(1 + r_{\text{total}})^{252/n} - 1$

**Risk Metrics:**
- **Annual Volatility**: $\sigma_{\text{annual}} = \sigma_{\text{daily}} \times \sqrt{252}$
- **Max Drawdown**: $\max_t \left(\frac{\text{RunningMax}_t - \text{Value}_t}{\text{RunningMax}_t}\right)$

**Risk-Adjusted Metrics:**
- **Sharpe Ratio**: $\frac{r_p - r_f}{\sigma_p}$ where $r_f = 3\%$
- **Sortino Ratio**: $\frac{r_p - r_f}{\sigma_{\text{downside}}}$ (only counts negative returns)
- **Calmar Ratio**: $\frac{r_p}{|\text{Max Drawdown}|}$

**Trading Metrics:**
- **Rebalances Executed**: Number of times portfolio was actually rebalanced
- **Rebalances Skipped**: Number of times rebalancing was skipped (v1.1, v1.2 only)
- **Total Commissions**: Sum of all transaction costs (¥)
- **Total Trades**: Number of individual buy/sell orders

### Implementation:
- **Comparison table**: `src/utils/reporting.py:40-109` (function `print_comparison_table`)
- **Metrics calculation**: `src/metrics.py:352-390` (function `calculate_all_metrics`)
- **Sharpe ratio**: `src/metrics.py:57-81`
- **Max drawdown**: `src/metrics.py:123-145`

### Before Moving On:
Review `src/metrics.py` to understand how each metric is calculated

In [ ]:
# Print comparison table
comparison_df = comparison.print_comparison()

## 6. Incremental Improvements

### What We're Doing:
Calculating and displaying the incremental benefit of each improvement.

### Analysis Structure:

**v1.0 → v1.1 (Adaptive Rebalancing Effect):**

$$\Delta_{\text{value}} = V_{1.1} - V_{1.0}$$
$$\Delta_{\text{return}} = r_{1.1} - r_{1.0}$$
$$\Delta_{\text{commission}} = C_{1.0} - C_{1.1}$$ (savings)

**v1.1 → v1.2 (Shrinkage Effect):**

$$\Delta_{\text{value}} = V_{1.2} - V_{1.1}$$
$$\Delta_{\text{return}} = r_{1.2} - r_{1.1}$$

**v1.0 → v1.2 (Combined Effect):**

$$\Delta_{\text{total}} = V_{1.2} - V_{1.0}$$

### Why This Matters:
- **v1.0 → v1.1**: Shows cost reduction impact (fewer unnecessary rebalances)
- **v1.1 → v1.2**: Shows estimation quality impact (better covariance → better weights)
- **v1.0 → v1.2**: Shows total improvement from both innovations

### Implementation:
- **Improvement calculation**: `src/comparison.py:235-271` (method `calculate_improvements`)
- **Display logic**: `src/comparison.py:193-232` (method `print_incremental_improvements`)
- **Formatting**: `src/utils/reporting.py:112-147` (function `print_improvement_summary`)

### Expected Results:
- v1.1 should show **modest improvement** (3-5%) from cost savings
- v1.2 should show **significant improvement** (20-30%) from better estimation

### Before Moving On:
Review `src/comparison.py:235-271` to understand improvement calculation logic

In [ ]:
# Print incremental improvements
comparison.print_incremental_improvements()

## 7. Equity Curves

### What We're Doing:
Visualizing portfolio value and drawdowns over time for all three versions.

### Top Panel: Equity Curves

**Formula**: At each time $t$:

$$V_t = \text{Cash}_t + \sum_{i=1}^{n} \text{Shares}_i \times \text{Price}_{i,t}$$

**What to Look For:**
- **Slope**: Steeper = higher returns
- **Separation**: How much versions diverge over time
- **Smoothness**: Fewer jumps = more stable

### Bottom Panel: Drawdowns

**Formula**: At each time $t$:

$$\text{DD}_t = \frac{V_t - \max_{s \leq t} V_s}{\max_{s \leq t} V_s}$$

**What to Look For:**
- **Depth**: How far below peak (more negative = worse)
- **Duration**: How long to recover from drawdown
- **Frequency**: How often drawdowns occur

### Implementation:
- **Equity curve extraction**: `src/comparison.py:181-184` (method `get_equity_curves`)
- **Equity tracking**: `src/strategy.py:118-120` (stored during backtest)
- **Portfolio valuation**: `src/portfolio.py:52-66` (method `get_value`)

### Interpretation:
- **v1.2 above v1.1 above v1.0**: Confirms incremental improvements
- **Divergence accelerates over time**: Compounding effect of better returns
- **Similar drawdown patterns**: All versions affected by same market events

### Before Moving On:
Review `src/portfolio.py:52-66` to understand how portfolio value is calculated

In [ ]:
# Get equity curves
equity_curves = comparison.get_equity_curves()
results = comparison.results

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

colors = {'v1.0': '#1f77b4', 'v1.1': '#ff7f0e', 'v1.2': '#2ca02c'}
labels = {'v1.0': 'v1.0 (Baseline)', 'v1.1': 'v1.1 (+Adaptive)', 'v1.2': 'v1.2 (+Shrinkage)'}

# Equity curves
for version, equity in equity_curves.items():
    ax1.plot(equity.index, equity, label=labels[version], 
             linewidth=2.5, alpha=0.8, color=colors[version])
    
    # Add final value annotation
    ax1.annotate(f'{version}: {format_currency(equity.iloc[-1])}',
                xy=(equity.index[-1], equity.iloc[-1]),
                xytext=(10, 0), textcoords='offset points',
                fontsize=9, color=colors[version], fontweight='bold')

ax1.set_title('Portfolio Equity Curves (2018-2026)', fontsize=14, fontweight='bold')
ax1.set_ylabel('Portfolio Value (¥)', fontsize=12)
ax1.legend(fontsize=11, loc='upper left')
ax1.grid(True, alpha=0.3)
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'¥{x/1e6:.2f}M'))

# Drawdowns
for version, equity in equity_curves.items():
    running_max = equity.expanding().max()
    drawdown = (equity - running_max) / running_max
    ax2.plot(drawdown.index, drawdown, label=labels[version], 
             linewidth=2, alpha=0.8, color=colors[version])

ax2.set_title('Drawdown Comparison', fontsize=14, fontweight='bold')
ax2.set_ylabel('Drawdown', fontsize=12)
ax2.set_xlabel('Date', fontsize=12)
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x:.0%}'))
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)

plt.tight_layout()
plt.show()

## 8. Performance Metrics Dashboard

### What We're Doing:
Creating a 6-panel visualization to compare key metrics across versions.

### Panel Descriptions:

**1. Annual Return** (Top-Left):
- Formula: $(1 + r_{\text{total}})^{252/n} - 1$
- **Higher is better**
- Shows compounded annualized growth rate

**2. Sharpe Ratio** (Top-Middle):
- Formula: $(r_p - 0.03) / \sigma_p$
- **Higher is better**
- Measures return per unit of risk
- >1.0 is good, >2.0 is excellent

**3. Max Drawdown** (Top-Right):
- Formula: $\max_t (\text{Peak}_t - \text{Trough}_t) / \text{Peak}_t$
- **Lower is better**
- Shows worst peak-to-trough decline

**4. Final Portfolio Value** (Bottom-Left):
- Direct measurement: $V_{\text{final}}$
- Shows terminal wealth from ¥1M initial

**5. Total Rebalances** (Bottom-Middle):
- Count of executed rebalances
- **v1.0**: Always rebalances (384)
- **v1.1/v1.2**: Adaptive (much fewer)

**6. Total Commissions** (Bottom-Right):
- Sum: $\sum_{\text{trades}} \text{TradeValue} \times 0.0003$
- Shows total transaction costs
- Directly impacts net returns

### Implementation:
- **Summary extraction**: `src/comparison.py:186-198` (method `get_summary_dict`)
- **Metrics aggregation**: `src/utils/reporting.py:250-272` (function `create_summary_dict`)

### What to Look For:
- **Return improvement**: v1.2 > v1.1 > v1.0
- **Sharpe improvement**: Better risk-adjusted returns
- **Rebalance reduction**: v1.1, v1.2 much fewer than v1.0
- **Trade-off**: v1.2 may have more rebalances than v1.1 but better returns justify it

### Before Moving On:
Review `src/utils/reporting.py:250-272` to see metric extraction logic

In [ ]:
# Get summary data
summaries = comparison.get_summary_dict()
versions = list(summaries.keys())

fig, axes = plt.subplots(2, 3, figsize=(16, 10))

# 1. Annual Returns
ax = axes[0, 0]
returns = [summaries[v]['annual_return'] * 100 for v in versions]
bars = ax.bar(versions, returns, color=[colors[v] for v in versions], alpha=0.8)
ax.set_title('Annual Return', fontsize=12, fontweight='bold')
ax.set_ylabel('Return (%)')
ax.grid(True, alpha=0.3, axis='y')
for bar, val in zip(bars, returns):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(),
            f'{val:.2f}%', ha='center', va='bottom', fontweight='bold')

# 2. Sharpe Ratios
ax = axes[0, 1]
sharpe = [summaries[v]['sharpe_ratio'] for v in versions]
bars = ax.bar(versions, sharpe, color=[colors[v] for v in versions], alpha=0.8)
ax.set_title('Sharpe Ratio', fontsize=12, fontweight='bold')
ax.set_ylabel('Sharpe Ratio')
ax.grid(True, alpha=0.3, axis='y')
for bar, val in zip(bars, sharpe):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(),
            f'{val:.2f}', ha='center', va='bottom', fontweight='bold')

# 3. Max Drawdown
ax = axes[0, 2]
dd = [abs(summaries[v]['max_drawdown']) * 100 for v in versions]
bars = ax.bar(versions, dd, color=[colors[v] for v in versions], alpha=0.8)
ax.set_title('Max Drawdown (absolute)', fontsize=12, fontweight='bold')
ax.set_ylabel('Drawdown (%)')
ax.grid(True, alpha=0.3, axis='y')
for bar, val in zip(bars, dd):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(),
            f'{val:.2f}%', ha='center', va='bottom', fontweight='bold')

# 4. Final Values
ax = axes[1, 0]
final_vals = [summaries[v]['final_value'] / 1e6 for v in versions]
bars = ax.bar(versions, final_vals, color=[colors[v] for v in versions], alpha=0.8)
ax.set_title('Final Portfolio Value', fontsize=12, fontweight='bold')
ax.set_ylabel('Value (¥M)')
ax.grid(True, alpha=0.3, axis='y')
for bar, val in zip(bars, final_vals):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(),
            f'¥{val:.2f}M', ha='center', va='bottom', fontweight='bold')

# 5. Rebalances
ax = axes[1, 1]
rebalances = [summaries[v]['rebalances'] for v in versions]
bars = ax.bar(versions, rebalances, color=[colors[v] for v in versions], alpha=0.8)
ax.set_title('Total Rebalances', fontsize=12, fontweight='bold')
ax.set_ylabel('Count')
ax.grid(True, alpha=0.3, axis='y')
for bar, val in zip(bars, rebalances):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(),
            f'{val}', ha='center', va='bottom', fontweight='bold')

# 6. Commissions
ax = axes[1, 2]
commissions = [summaries[v]['commissions'] for v in versions]
bars = ax.bar(versions, commissions, color=[colors[v] for v in versions], alpha=0.8)
ax.set_title('Total Commissions', fontsize=12, fontweight='bold')
ax.set_ylabel('Commissions (¥)')
ax.grid(True, alpha=0.3, axis='y')
for bar, val in zip(bars, commissions):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(),
            f'¥{val:,.0f}', ha='center', va='bottom', fontweight='bold', fontsize=9)

plt.tight_layout()
plt.show()

## 9. Risk-Return Analysis

### What We're Doing:
Plotting each version in risk-return space with Sharpe ratio reference lines.

### Coordinate System:
- **X-axis**: Annual Volatility ($\sigma_p$)
- **Y-axis**: Annual Return ($r_p$)
- **Diagonal lines**: Constant Sharpe ratios

### Sharpe Ratio Lines:

Points on line satisfy: $r = \text{Sharpe} \times \sigma$

**Interpretation**:
- Points **above** a Sharpe line have **higher** Sharpe than that line
- Points **below** a Sharpe line have **lower** Sharpe than that line
- **Upper-left** is ideal: High return, low risk

### What to Look For:
1. **Version positions**: v1.2 should be highest (best return) and leftmost (lowest risk)
2. **Sharpe line comparison**: Higher Sharpe = steeper slope
3. **Risk-return trade-off**: Is extra return worth extra risk?

### Expected Pattern:
```
v1.2 •
        
v1.1   •
        
v1.0     •
────────────→ Volatility
```

### Implementation:
- **Risk-return extraction**: From `summaries` dict created in previous cell
- **Sharpe calculation**: `src/metrics.py:57-81`

### Before Moving On:
Review `src/metrics.py:57-81` to understand Sharpe ratio calculation

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

# Extract metrics
vols = [summaries[v]['annual_volatility'] * 100 for v in versions]
rets = [summaries[v]['annual_return'] * 100 for v in versions]

# Plot points
for i, version in enumerate(versions):
    ax.scatter(vols[i], rets[i], s=500, alpha=0.7, color=colors[version], 
               label=labels[version], edgecolors='black', linewidth=2)
    ax.annotate(version, (vols[i], rets[i]), xytext=(10, 10), 
                textcoords='offset points', fontsize=12, 
                fontweight='bold', color=colors[version])

# Add Sharpe ratio reference lines
max_vol = max(vols) * 1.1
for sharpe_val in [0.5, 1.0, 1.5]:
    ax.plot([0, max_vol], [0, max_vol * sharpe_val], 'k--', alpha=0.2, linewidth=1)
    ax.text(max_vol * 0.9, max_vol * sharpe_val * 0.9, f'Sharpe={sharpe_val}',
            fontsize=9, alpha=0.5, rotation=np.degrees(np.arctan(sharpe_val)))

ax.set_title('Risk-Return Profile (2018-2026)', fontsize=14, fontweight='bold')
ax.set_xlabel('Annual Volatility (%)', fontsize=12)
ax.set_ylabel('Annual Return (%)', fontsize=12)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_xlim(left=0)
ax.set_ylim(bottom=0)

plt.tight_layout()
plt.show()

## 10. Covariance Shrinkage Analysis

### What We're Doing:
Deep dive into the v1.2 innovation - Ledoit-Wolf covariance shrinkage.

### The Problem with Sample Covariance:

With limited data (252 days), sample covariance $\hat{\Sigma}$ is **noisy**:
- Overestimates extreme correlations
- Underestimates moderate correlations
- Leads to unstable portfolio weights

### Ledoit-Wolf Shrinkage Solution:

$$\Sigma_{\text{shrunk}} = (1 - \delta) \hat{\Sigma} + \delta F$$

Where:
- $\hat{\Sigma}$ = Sample covariance
- $F$ = Structured target (constant correlation model)
- $\delta \in [0, 1]$ = Shrinkage intensity (optimally chosen)

**Key Insight**: Shrinkage **pulls extreme estimates toward average**, reducing estimation error.

### Three Heatmaps:

**1. Sample Covariance** (Left):
- Raw empirical covariance from 252 days
- Used by v1.0 and v1.1
- Noisy, especially off-diagonal elements

**2. Shrunk Covariance** (Middle):
- Ledoit-Wolf shrinkage applied
- Used by v1.2
- Smoother, more stable

**3. Difference** (Right):
- Shrunk - Sample
- Shows what changed
- Extreme values pulled toward mean

### Shrinkage Coefficient ($\delta$):

$$\delta^* = \arg\min_{\delta} E[||\Sigma_{\text{shrunk}} - \Sigma_{\text{true}}||^2]$$

Analytically solved by Ledoit-Wolf (2004).

**Typical range**: 0.1-0.4
- Low $\delta$ (0.1): Sample is good enough, little shrinkage needed
- High $\delta$ (0.4): Sample is very noisy, more shrinkage needed

### Implementation:
- **Shrinkage estimation**: `src/optimizer.py:15-30` (function `estimate_covariance_shrinkage`)
- **Uses**: `sklearn.covariance.LedoitWolf`
- **Called from**: `src/optimizer.py:51-56` when `use_shrinkage=True`

### Impact on Weights:

Shrunk covariance → More stable weights → Better out-of-sample performance

### Before Moving On:
1. Read `src/optimizer.py:15-30` to see shrinkage implementation
2. Review `src/optimizer.py:48-56` to see how shrinkage integrates with optimization
3. (Optional) Read Ledoit & Wolf (2004) paper for mathematical details

In [ ]:
# Demonstrate shrinkage on recent data
recent_returns = prices.tail(252).pct_change().dropna()

# Sample covariance (v1.0, v1.1)
sample_cov = recent_returns.cov()

# Shrunk covariance (v1.2)
shrunk_cov, shrinkage_coef = estimate_covariance_shrinkage(recent_returns)
shrunk_cov_df = pd.DataFrame(shrunk_cov, index=sample_cov.index, columns=sample_cov.columns)

print_section("Covariance Shrinkage Analysis")
print(f"\nShrinkage coefficient: {shrinkage_coef:.4f}")
print(f"Interpretation: {(1-shrinkage_coef)*100:.1f}% sample cov + {shrinkage_coef*100:.1f}% structured\n")

# Compare volatilities
vol_comparison = pd.DataFrame({
    'Sample': np.sqrt(np.diag(sample_cov)) * np.sqrt(252) * 100,
    'Shrunk': np.sqrt(np.diag(shrunk_cov)) * np.sqrt(252) * 100
}, index=sample_cov.index)
vol_comparison['Diff'] = vol_comparison['Shrunk'] - vol_comparison['Sample']

print("Asset Volatilities (%):\n")
print(vol_comparison)

# Visualize covariance matrices
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16, 5))

vmin, vmax = min(sample_cov.min().min(), shrunk_cov_df.min().min()), max(sample_cov.max().max(), shrunk_cov_df.max().max())

# Sample covariance
im1 = ax1.imshow(sample_cov, cmap='RdBu_r', aspect='auto', vmin=vmin, vmax=vmax)
ax1.set_title('Sample Covariance\n(v1.0, v1.1)', fontweight='bold')
ax1.set_xticks(range(len(sample_cov.columns)))
ax1.set_yticks(range(len(sample_cov.index)))
ax1.set_xticklabels(sample_cov.columns, rotation=45, ha='right')
ax1.set_yticklabels(sample_cov.index)
plt.colorbar(im1, ax=ax1, fraction=0.046, pad=0.04)

# Shrunk covariance
im2 = ax2.imshow(shrunk_cov_df, cmap='RdBu_r', aspect='auto', vmin=vmin, vmax=vmax)
ax2.set_title(f'Shrunk Covariance\n(v1.2, λ={shrinkage_coef:.3f})', fontweight='bold')
ax2.set_xticks(range(len(shrunk_cov_df.columns)))
ax2.set_yticks(range(len(shrunk_cov_df.index)))
ax2.set_xticklabels(shrunk_cov_df.columns, rotation=45, ha='right')
ax2.set_yticklabels(shrunk_cov_df.index)
plt.colorbar(im2, ax=ax2, fraction=0.046, pad=0.04)

# Difference
diff = shrunk_cov_df - sample_cov
diff_max = diff.abs().max().max()
im3 = ax3.imshow(diff, cmap='RdBu_r', aspect='auto', vmin=-diff_max, vmax=diff_max)
ax3.set_title('Difference\n(Shrunk - Sample)', fontweight='bold')
ax3.set_xticks(range(len(diff.columns)))
ax3.set_yticks(range(len(diff.index)))
ax3.set_xticklabels(diff.columns, rotation=45, ha='right')
ax3.set_yticklabels(diff.index)
plt.colorbar(im3, ax=ax3, fraction=0.046, pad=0.04)

plt.tight_layout()
plt.show()

print("\n💡 Key Insight: Shrinkage reduces extreme off-diagonal elements,")
print("               leading to more stable and robust weight estimates.")

## 11. Weight Evolution Analysis

### What We're Doing:
Visualizing how portfolio weights change over time for each version.

### Weight Calculation at Each Rebalance:

For each version, weights are optimized to achieve equal risk contribution:

$$w^* = \arg\min_{w} \text{std}(RC_1, ..., RC_n)$$

subject to: $\sum_i w_i = 1, \quad w_i \geq 0$

### What Causes Weight Changes:

1. **Changing correlations**: Assets become more/less correlated
2. **Changing volatilities**: Asset risk profiles shift
3. **Market movements**: Price changes alter current weights (drift)

### Three Panels (v1.0, v1.1, v1.2):

**v1.0 Panel** (Top):
- 384 rebalances (every Monday)
- Weights change frequently
- Responds to every market fluctuation

**v1.1 Panel** (Middle):
- Only 5 rebalances (adaptive)
- Weights very stable
- Only rebalances when drift > 5%
- **Trade-off**: Saves costs but may deviate from optimal

**v1.2 Panel** (Bottom):
- 175 rebalances (more than v1.1, less than v1.0)
- **Why?** Shrinkage changes optimal weights more
- Smoother transitions than v1.0
- Better weight stability than raw sample covariance

### Reading the Stacked Area Chart:

- **Y-axis**: Portfolio weight (0-100%)
- **Color bands**: Each asset's allocation over time
- **Band width**: Larger = more allocation
- **Band changes**: Rebalancing events

### Expected Pattern:

- **Bonds** (511260.SH): Usually largest (60-80%)
  - Why? Bonds have lowest volatility, need high weight for equal risk
- **Stocks**: Moderate allocation (15-25%)
  - Why? Higher volatility, smaller weight needed
- **Gold/Commodities**: Smaller allocation (5-15%)

### Implementation:
- **Weight storage**: `src/strategy.py:148` (stored during rebalancing)
- **Weight optimization**: `src/optimizer.py:33-78`
- **Weight tracking**: `src/comparison.py` stores in `results['weights_history']`

### Key Observation:

If v1.2 shows smoother weight evolution than v1.0, shrinkage is working!

### Before Moving On:
Review `src/strategy.py:134-148` to see weight storage logic

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(14, 12))

for i, version in enumerate(versions):
    weights_history = results[version]['weights_history']
    
    if not weights_history.empty:
        weights_history.plot.area(stacked=True, ax=axes[i], alpha=0.7, 
                                 legend=(i == 2))  # Only show legend on last plot
        
        rebal_count = results[version]['rebalances_executed']
        axes[i].set_title(f'{version} Weight Evolution ({rebal_count} rebalances)',
                         fontsize=12, fontweight='bold')
        axes[i].set_ylabel('Weight')
        axes[i].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x:.0%}'))
        axes[i].grid(True, alpha=0.3)
        
        if i == 2:  # Last plot
            axes[i].legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=9)
            axes[i].set_xlabel('Date')

plt.tight_layout()
plt.show()

## 12. Summary & Recommendations

### What We're Doing:
Generating a comprehensive summary with version ranking, insights, and recommendations.

### Components:

**1. Version Ranking**:
- Sorts versions by Sharpe ratio (risk-adjusted returns)
- Shows which version performed best

**2. Key Insights**:
- Quantifies improvement of each innovation
- v1.0 → v1.1: Cost reduction (adaptive rebalancing)
- v1.1 → v1.2: Estimation quality (shrinkage)

**3. Recommendation**:
- Best version for production use
- Justification based on metrics

### Implementation:
- **Summary generation**: `src/comparison.py:273-290` (method `print_summary`)
- **Ranking**: `src/utils/reporting.py:150-181` (function `print_version_ranking`)
- **Insights**: `src/utils/reporting.py:184-207` (function `print_key_insights`)

### Expected Output:

```
Version Ranking (by Sharpe Ratio):
  🥇 v1.2: Sharpe 1.34, Final Value ¥2,191,500
  🥈 v1.1: Sharpe 1.13, Final Value ¥1,764,973
  🥉 v1.0: Sharpe 1.11, Final Value ¥1,697,909

Key Insights:
  • v1.0 → v1.1: +¥67,064 (+3.9%)
    → Adaptive rebalancing reduces costs
  • v1.1 → v1.2: +¥426,527 (+24.2%)
    → Covariance shrinkage improves robustness

Recommendation: Use v1.2 for production trading
```

### Decision Criteria:

Choose **v1.2** if:
- ✓ You want best returns
- ✓ You want best risk-adjusted returns
- ✓ You can accept moderate rebalancing frequency

Choose **v1.1** if:
- ✓ You want minimal transaction costs
- ✓ You want simplest implementation (no shrinkage)
- ✓ You can accept lower returns

Choose **v1.0** if:
- ✓ You need academic baseline
- ✓ You want purest risk parity implementation
- ✓ Transaction costs are negligible

### Before Moving On:
Review `src/comparison.py:273-290` to understand summary generation logic

In [ ]:
# Print comprehensive summary
comparison.print_summary()

## 13. Generate Report

### What We're Doing:
Creating a text-based summary report that can be exported or saved.

### Report Contents:

1. **Header**: Period and initial capital
2. **Version summaries**: Key metrics for each version
3. **Performance comparison**: Side-by-side comparison

### Use Cases:
- Save to file for documentation
- Include in research reports
- Share with stakeholders
- Archive for future reference

### Implementation:
- **Report generation**: `src/comparison.py:292-320` (method `generate_report`)
- **Formatting**: Plain text format for maximum compatibility

### Example Output Structure:

```
======================================================================
ALL WEATHER STRATEGY - VERSION COMPARISON REPORT
======================================================================

Period: 2018-01-01 to 2026-01-28
Initial Capital: ¥1,000,000

v1.0: Pure Risk Parity (Always Rebalance, Sample Covariance)
  Final Value: ¥1,697,909
  Annual Return: 7.05%
  Sharpe Ratio: 1.11
  ...
```

### To Save Report:

```python
report = comparison.generate_report()
with open('version_comparison_report.txt', 'w') as f:
    f.write(report)
```

### Before Moving On:
Review `src/comparison.py:292-320` to see report formatting logic

In [ ]:
# Generate and print text report
report = comparison.generate_report()
print(report)

## Conclusion

This notebook demonstrates the **evolution of the All Weather Strategy** through three versions:

### Version Evolution:

**v1.0 → v1.1 (Adaptive Rebalancing)**:
- **Improvement**: +3.9% final value
- **Mechanism**: Skip rebalancing when drift < 5%
- **Benefit**: 72% commission savings
- **Trade-off**: Slight deviation from optimal weights
- **Formula**: $\max_i |w_i^{\text{curr}} - w_i^{\text{target}}| > 0.05 \Rightarrow \text{rebalance}$

**v1.1 → v1.2 (Covariance Shrinkage)**:
- **Improvement**: +24.2% final value
- **Mechanism**: Ledoit-Wolf shrinkage for covariance estimation
- **Benefit**: More stable, robust weights
- **Trade-off**: More rebalances than v1.1 (but justified by returns)
- **Formula**: $\Sigma = (1-\delta)\hat{\Sigma} + \delta F$

**v1.0 → v1.2 (Combined)**:
- **Total improvement**: +29.1% final value
- **Synergy**: Adaptive rebalancing + robust estimation

### Key Result:

v1.2 achieves **10.62% annual return** with **1.34 Sharpe ratio**, representing a **29% improvement** over v1.0 baseline.

### Mathematical Summary:

$$\text{v1.2} = \text{Risk Parity} + \text{Adaptive} + \text{Shrinkage}$$

$$w^* = \arg\min_w \text{std}(RC_1, ..., RC_n)$$

Where:
- $RC_i = w_i (\Sigma w)_i / \sigma_p$ computed with shrunk $\Sigma$
- Rebalancing only when drift > 5%

### Configuration for Production Use

```python
# Recommended: v1.2 with adaptive rebalancing + shrinkage
from src.strategy import AllWeatherV1

strategy = AllWeatherV1(
    prices=prices,
    initial_capital=1_000_000,
    rebalance_freq='W-MON',           # Weekly Monday
    lookback=252,                     # 1 year
    commission_rate=0.0003,           # 0.03%
    rebalance_threshold=0.05,         # 5% drift → rebalance
    use_shrinkage=True                # Ledoit-Wolf
)
```

### Further Reading:

**Code References**:
- Risk parity optimization: `src/optimizer.py:33-78`
- Adaptive rebalancing: `src/strategy.py:61-90`
- Covariance shrinkage: `src/optimizer.py:15-30`
- Portfolio management: `src/portfolio.py`
- Performance metrics: `src/metrics.py`

**Academic Papers**:
- Ledoit & Wolf (2004): "A well-conditioned estimator for large-dimensional covariance matrices"
- Maillard et al. (2010): "The properties of equally weighted risk contribution portfolios"
- Asness et al. (2012): "Leverage aversion and risk parity"

**Project Documentation**:
- `CLAUDE.md`: Full strategy documentation
- `docs/RISK_PARITY_ANALYSIS.md`: Mathematical analysis
- `V1.3_4QUADRANT_SUMMARY.md`: Advanced extensions